In [1]:
%load_ext autoreload
%autoreload 2
   
from clfm import clfm
import matplotlib.pyplot as plt

In [2]:
# path to the database root folder
path_database = '/media/elea/ssd/CLFM_DB/polarimetry'

# possible options include "polarimetry", "histology" and "custom"
# for custom, you should also provide the threshold (maximal distance, in pixel, for correct match) and 
# mm_per_pixel (the spatial resolution of your images)
type_alignment = 'polarimetry'

# create the project and load the image pairs
comparer = clfm.CLFMProject(path_database, type_alignment)
comparer.prepare_imgs()

# Run the different methods and save the coordinates

### 1. SIFT + MNN

In [4]:
sift_mnn_matcher = clfm.Matcher('sift', 'mnn', repetitions=comparer.repetitions)
sift_mnn_matcher.run(comparer.parameters_dict, comparer.images)

Local feature matching: 100%|██████████████████████████████████████████████████████████| 15/15 [00:44<00:00,  2.94s/it]

Processing time: 2.48 ± 0.06 s


### 2. SiLK + MNN

In [5]:
silk_mnn_matcher = clfm.Matcher('silk', 'mnn', repetitions=comparer.repetitions)
silk_mnn_matcher.run(comparer.parameters_dict, comparer.images)

Local feature matching: 100%|██████████████████████████████████████████████████████████| 15/15 [00:16<00:00,  1.11s/it]

Processing time: 0.69 ± 0.03 s


### 3. Superpoint + MNN

In [6]:
sp_mnn_matcher = clfm.Matcher('superpoint', 'mnn', repetitions=comparer.repetitions)
sp_mnn_matcher.run(comparer.parameters_dict, comparer.images)

Local feature matching: 100%|██████████████████████████████████████████████████████████| 15/15 [01:03<00:00,  4.20s/it]

Processing time: 3.77 ± 0.04 s


### 4. SIFT + OANet

In [7]:
sift_oanet_matcher = clfm.Matcher('sift', 'oanet', repetitions=comparer.repetitions)
sift_oanet_matcher.run(comparer.parameters_dict, comparer.images)

Local feature matching: 100%|██████████████████████████████████████████████████████████| 15/15 [01:23<00:00,  5.57s/it]

Processing time: 5.16 ± 0.07 s


### 5. SiLK + OANet

In [8]:
silk_oanet_matcher = clfm.Matcher('silk', 'oanet', repetitions=comparer.repetitions)
silk_oanet_matcher.run(comparer.parameters_dict, comparer.images)

Local feature matching: 100%|██████████████████████████████████████████████████████████| 15/15 [00:51<00:00,  3.45s/it]

Processing time: 3.01 ± 0.32 s


### 6. Superpoint + OANet

In [9]:
sp_oanet_matcher = clfm.Matcher('superpoint', 'oanet', repetitions=comparer.repetitions)
sp_oanet_matcher.run(comparer.parameters_dict, comparer.images)

Local feature matching: 100%|██████████████████████████████████████████████████████████| 15/15 [01:56<00:00,  7.79s/it]

Processing time: 7.34 ± 0.44 s


### 7. SP + SG

In [10]:
sp_sg_matcher = clfm.Matcher('superpoint', 'superglue', repetitions=comparer.repetitions)
sp_sg_matcher.run(comparer.parameters_dict, comparer.images)

Local feature matching: 100%|██████████████████████████████████████████████████████████| 15/15 [02:01<00:00,  8.09s/it]

Processing time: 7.68 ± 0.17 s


### 8. Omniglue

In [4]:
omniglue_matcher = clfm.Matcher('omniglue', 'omniglue', repetitions=comparer.repetitions)
omniglue_matcher.run(comparer.parameters_dict, comparer.images)

Local feature matching: 100%|██████████████████████████████████████████████████████████| 15/15 [08:27<00:00, 33.85s/it]

Processing time: 33.41 ± 0.86 s


# Align the images using bUnwarpJ and the previously obtained coordinates

In [3]:
combinations = (
    ('sift', 'mnn'),
    ('silk', 'mnn'),
    ('superpoint', 'mnn'),
    ('sift', 'oanet'),
    ('silk', 'oanet'),
    ('superpoint', 'oanet'),
    ('superpoint', 'superglue'),
    ('omniglue', 'omniglue')
)

In [4]:
from clfm import register, metrics

# if the ground truth maps have been generated manually
if type_alignment != 'histology':
    register.register_images(
        comparer.parameters_dict,
        ['gt']
    )
    pass
else:
    combinations = (*combinations, ('manual'))

Aligning images: 100%|█████████████████████████| 15/15 [00:00<00:00, 111.78it/s]


In [5]:
rmses, precisions = metrics.find_rmse(
    combinations,
    comparer.parameters_dict,
    comparer.images,
    comparer.type_alignment,
    comparer.threshold
)

Calculating RMSE: 100%|█████████████████████████| 15/15 [00:30<00:00,  2.00s/it]


In [6]:
rmses_agg, precisions_agg = metrics.aggregate_and_bootstrap_metrics(
    comparer.parameters_dict,
    combinations,
    type_alignment,
    comparer.threshold,
    comparer.mm_per_pixel,
    num_samples=1000
)

metrics.print_bootstrap_results(
    rmses_agg,
    precisions_agg,
)

Bootstrapping metrics: 100%|██████████████████████| 8/8 [00:06<00:00,  1.17it/s]

Combination               |   RMSE(px) ± error      |   RMSE(mm) ± error      |  Prec(%) ± error   
-----------------------------------------------------------------------------------------------
sift + mnn                |     246.33 ± 2.64       |      12.32 ± 0.13       |     3.60 ± 0.43    
silk + mnn                |     243.69 ± 2.46       |      12.18 ± 0.12       |     6.20 ± 0.51    
superpoint + mnn          |     127.01 ± 7.64       |       6.35 ± 0.38       |    61.93 ± 2.50    
sift + oanet              |     152.87 ± 5.81       |       7.64 ± 0.29       |    24.90 ± 1.81    
silk + oanet              |     104.48 ± 2.65       |       5.22 ± 0.13       |    43.00 ± 0.93    
superpoint + oanet        |       6.56 ± 0.48       |       0.33 ± 0.02       |    97.28 ± 1.15    
superpoint + superglue    |       6.32 ± 0.68       |       0.32 ± 0.03       |    97.53 ± 1.23    
omniglue + omniglue       |      13.07 ± 2.21       |       0.65 ± 0.11       |    90.21 ± 1.80    
